# Relative Value Credit Analysis

This notebook explores the features of the relative value credit analysis framework. We will walk through the full pipeline, from data loading to backtesting, for two pairs of credit default swap (CDS) indices.

## 1. Load and Prepare Data

First, we load the data for our two pairs of tickers. We use the `data_agent` to handle loading the CSV files, cleaning them by forward-filling missing values, and synchronizing the time series to a common date index. We'll use the following pairs:

- **Pair 1:** `CDXTEL15` and `CDXTHL15`
- **Pair 2:** `ITRXTE5I` and `ITRXTEXM`

In [ ]:
import sys
import os
# Add src to the python path
sys.path.append(os.path.abspath('src'))

import data_agent as da

# --- Pair 1 ---
ticker_x1 = 'CDXTEL15'
ticker_y1 = 'CDXTHL15'

# Load data
data_x1 = da.load_data(ticker_x1)
data_y1 = da.load_data(ticker_y1)

# Clean data
data_x1_clean = da.clean_data(data_x1)
data_y1_clean = da.clean_data(data_y1)

# Synchronize data
sync_x1, sync_y1 = da.synchronize_data(data_x1_clean, data_y1_clean)

# --- Pair 2 ---
ticker_x2 = 'ITRXTE5I'
ticker_y2 = 'ITRXTEXM'

# Load data
data_x2 = da.load_data(ticker_x2)
data_y2 = da.load_data(ticker_y2)

# Clean data
data_x2_clean = da.clean_data(data_x2)
data_y2_clean = da.clean_data(data_y2)

# Synchronize data
sync_x2, sync_y2 = da.synchronize_data(data_x2_clean, data_y2_clean)

print("Data loaded and prepared for both pairs.")
print("Pair 1 shapes:", sync_x1.shape, sync_y1.shape)
print("Pair 2 shapes:", sync_x2.shape, sync_y2.shape)

## 2. Run Modeling Pipeline

Next, we run the modeling pipeline on our prepared data. This involves two main steps:

1.  **Linear Regression:** We calculate the `beta` (hedge ratio) and `residuals` between the two series over a long-term lookback window.
2.  **Ornstein-Uhlenbeck (OU) Process:** We fit an OU process to the residuals to model their mean-reverting behavior. This gives us the parameters `theta` (speed of reversion), `mu` (long-term mean), and `sigma` (volatility).

We use different lookback parameters for each pair to demonstrate flexibility.

In [ ]:
import modeling_agent as ma

# --- Pair 1 Modeling ---
regression_lookback1 = '2Y'
ou_lookback1 = '26W'

beta1, residuals1, theta1, mu1, sigma1 = ma.run_modeling_pipeline(
    sync_x1['PX_LAST'],
    sync_y1['PX_LAST'],
    regression_lookback1,
    ou_lookback1
)

# --- Pair 2 Modeling ---
regression_lookback2 = '3Y'
ou_lookback2 = '52W'

beta2, residuals2, theta2, mu2, sigma2 = ma.run_modeling_pipeline(
    sync_x2['PX_LAST'],
    sync_y2['PX_LAST'],
    regression_lookback2,
    ou_lookback2
)

print("--- Pair 1 Results ---")
print(f"Beta: {beta1:.4f}")
print(f"Theta: {theta1:.4f}")
print(f"Mu: {mu1:.4f}")
print(f"Sigma: {sigma1:.4f}")
print(f"Is Mean Reverting: {ma.is_mean_reverting(theta1)}")

print("\n--- Pair 2 Results ---")
print(f"Beta: {beta2:.4f}")
print(f"Theta: {theta2:.4f}")
print(f"Mu: {mu2:.4f}")
print(f"Sigma: {sigma2:.4f}")
print(f"Is Mean Reverting: {ma.is_mean_reverting(theta2)}")

## 3. Generate Signals

Now we generate trading signals from the modeling results. The signal is essentially a z-score of the current residual, normalized by the equilibrium volatility of the OU process. This tells us how far the current relationship has deviated from its long-term mean.

In [ ]:
import signal_agent as sa
import pandas as pd

# --- Pair 1 Signal ---
eq_vol1 = sa.calculate_equilibrium_volatility(theta1, sigma1)
signals1 = residuals1.apply(lambda r: sa.generate_signal(r, mu1, eq_vol1))

# --- Pair 2 Signal ---
eq_vol2 = sa.calculate_equilibrium_volatility(theta2, sigma2)
signals2 = residuals2.apply(lambda r: sa.generate_signal(r, mu2, eq_vol2))

print("--- Pair 1 Signal ---")
print(f"Equilibrium Volatility: {eq_vol1:.4f}")
print("Last 5 signals:")
print(signals1.tail())

print("\n--- Pair 2 Signal ---")
print(f"Equilibrium Volatility: {eq_vol2:.4f}")
print("Last 5 signals:")
print(signals2.tail())

## 4. Run Backtest and Visualize

Finally, we run a simple backtest to evaluate the performance of our signals and visualize the results. The backtest will long the spread when the signal is below a certain threshold (e.g., -1.5) and short it when the signal is above the threshold (e.g., 1.5). We'll also generate plots for:

-   The time series of the residuals.
-   The residuals with the fitted OU mean.
-   The historical trading signals.

In [ ]:
import backtest_agent as ba

# --- Pair 1 Backtest ---
backtest_results1 = ba.run_backtest(sync_y1['PX_LAST'], signals1, signal_threshold=1.5)
print("--- Pair 1 Backtest Results ---")
for metric, value in backtest_results1.items():
    print(f"{metric.replace('_', ' ').title()}: {value:.4f}")

# --- Pair 2 Backtest ---
backtest_results2 = ba.run_backtest(sync_y2['PX_LAST'], signals2, signal_threshold=1.5)
print("\n--- Pair 2 Backtest Results ---")
for metric, value in backtest_results2.items():
    print(f"{metric.replace('_', ' ').title()}: {value:.4f}")

# --- Visualizations ---
# Pair 1
ba.plot_residuals(residuals1, 'residuals1.png')
ba.plot_ou_fit(residuals1, mu1, 'ou_fit1.png')
ba.plot_signal(signals1, 'signal1.png')

# Pair 2
ba.plot_residuals(residuals2, 'residuals2.png')
ba.plot_ou_fit(residuals2, mu2, 'ou_fit2.png')
ba.plot_signal(signals2, 'signal2.png')

print("\nVisualizations saved to files.")